<a href="https://colab.research.google.com/github/Nastyadimples/matching/blob/main/matching.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
pip install faiss-cpu

# Мэтчинг товаров

## Анализ данных

### Задача
* разработать алгоритм, который для всех товаров из validation.csv предложит несколько вариантов наиболее похожих товаров из base;
* оценить качество алгоритма по метрике accuracy@5

### Исходные данные
* base.csv - анонимизированный набор товаров. Каждый товар представлен как уникальный id (0-base, 1-base, 2-base) и вектор признаков размерностью 72.

* train.csv - обучающий датасет. Каждая строчка - один товар, для которого известен уникальный id (0-query, 1-query, …) , вектор признаков И id товара из base.csv, который максимально похож на него (по мнению экспертов).

* validation.csv - датасет с товарами (уникальный id и вектор признаков), для которых надо найти наиболее близкие товары из base.csv

* validation_answer.csv - правильные ответы к validation.csv.

In [2]:
#импорт библиотек
import pandas as pd
import numpy as np
import faiss
from sklearn.preprocessing import StandardScaler
from sklearn.decomposition import PCA

In [3]:
# чтение файлов с данными
df_base = pd.read_csv('base.csv', index_col=0)
df_tr = pd.read_csv('train.csv', index_col=0)
df_val = pd.read_csv('validation.csv', index_col=0)
val_ans = pd.read_csv('validation_answer.csv', index_col=0)

<ipython-input-3-2e846825267c>:2: DtypeWarning: Columns (11) have mixed types. Specify dtype option on import or set low_memory=False.
  df_base = pd.read_csv('base.csv', index_col=0)


In [4]:
#изучение данных
def greeting(df):
    display(df.head())
    display(df.describe())
    display(df.info())
    print('Количество дубликатов:', df.duplicated().sum())

In [5]:
greeting(df_base)

,0,1,2,3,4,5,6,7,8,9,...,62,63,64,65,66,67,68,69,70,71
Id,,,,,,,,,,,,,,,,,,,,,
0-base,-115.083890,11.152912,-64.426760,-118.88089,216.482440,-104.698060,-469.070588,44.348083,120.915344,181.449700,...,-42.808693,38.800827,-151.76218,-74.389090,63.66634,-4.703861,92.933610,115.269190,-112.756640,-60.830353
1-base,-34.562202,13.332763,-69.787610,-166.53348,57.680607,-86.098370,-85.076666,-35.637436,119.718636,195.234190,...,-117.767525,41.100000,-157.82940,-94.446806,68.20211,24.346846,179.937930,116.834000,-84.888941,-59.524610
2-base,-54.233746,6.379371,-29.210136,-133.41383,150.895830,-99.435326,52.554795,62.381706,128.951450,164.381470,...,-76.397800,46.011803,-207.14442,127.325570,65.56618,66.325680,81.073490,116.594154,-1074.464888,-32.527206
3-base,-87.520130,4.037884,-87.803030,-185.06763,76.369540,-58.985165,-383.182845,-33.611237,122.031910,136.233580,...,-70.647940,-6.358921,-147.20105,-37.692750,66.20289,-20.566910,137.206940,117.474100,-1074.464888,-72.915490
4-base,-72.743850,6.522049,43.671265,-140.60803,5.820023,-112.074080,-397.711282,45.182500,122.167180,112.119064,...,-57.199104,56.642403,-159.35184,85.944724,66.76632,-2.505783,65.315285,135.051590,-1074.464888,0.319401


,0,1,2,3,4,5,6,7,8,9,...,62,63,64,65,66,67,68,69,70,71
count,365294.000000,365294.000000,365294.000000,365294.000000,365294.000000,365294.000000,365294.000000,365294.000000,365294.000000,365294.000000,...,365293.000000,365293.000000,365293.000000,365293.000000,365293.000000,365293.000000,365293.000000,365293.000000,365293.000000,365293.000000
mean,-85.938253,7.935616,-44.363599,-146.485524,111.410494,-72.489551,-404.017180,20.633695,123.500888,124.787825,...,-79.647130,34.485176,-154.101611,14.021718,67.792157,23.570092,74.585873,115.514782,-770.807322,-47.979798
std,25.214515,4.961875,38.825561,20.065301,46.815340,28.340073,274.301191,64.861744,6.412827,64.611541,...,30.523313,27.794025,41.236400,98.993612,1.828296,55.335241,61.598702,21.349777,393.928922,41.604101
min,-193.805630,-12.712713,-237.128420,-228.158870,-88.206740,-184.152360,-791.468604,-264.100800,93.183750,-159.981660,...,-213.761960,-84.798650,-338.454280,-157.593866,60.350810,-207.154080,-203.601620,20.995232,-1297.931458,-226.714780
25%,-103.132303,4.555616,-69.564791,-160.017952,80.032188,-91.982571,-652.504013,-22.259680,119.208581,81.743813,...,-99.604860,17.891811,-180.370270,-71.657183,66.571570,-12.723154,33.249454,101.321110,-1074.464888,-75.881850
50%,-85.960822,7.892806,-43.617124,-146.604090,111.932105,-72.531200,-442.360948,21.099108,123.654652,124.224575,...,-79.226930,35.340195,-153.336350,13.968517,67.812904,23.483760,74.412860,115.853030,-987.795417,-48.591730
75%,-68.681810,11.309164,-19.166755,-132.998638,142.856313,-52.937550,-165.703140,64.537060,127.861492,167.895460,...,-59.062170,52.760056,-126.426110,99.595586,69.031265,60.003990,115.568600,129.723890,-448.914676,-19.833178
max,17.156853,27.362547,152.804410,-51.374780,308.063320,56.926674,109.632533,301.094240,152.261200,420.835300,...,60.174110,147.901660,14.601700,185.096144,74.900150,267.879460,339.573850,210.120960,98.768579,119.349030


<class 'pandas.core.frame.DataFrame'>
Index: 365294 entries, 0-base to 394209-base
Data columns (total 72 columns):
 #   Column  Non-Null Count   Dtype  
---  ------  --------------   -----  
 0   0       365294 non-null  float64
 1   1       365294 non-null  float64
 2   2       365294 non-null  float64
 3   3       365294 non-null  float64
 4   4       365294 non-null  float64
 5   5       365294 non-null  float64
 6   6       365294 non-null  float64
 7   7       365294 non-null  float64
 8   8       365294 non-null  float64
 9   9       365294 non-null  float64
 10  10      365294 non-null  object 
 11  11      365293 non-null  float64
 12  12      365293 non-null  float64
 13  13      365293 non-null  float64
 14  14      365293 non-null  float64
 15  15      365293 non-null  float64
 16  16      365293 non-null  float64
 17  17      365293 non-null  float64
 18  18      365293 non-null  float64
 19  19      365293 non-null  float64
 20  20      365293 non-null  float64
 21  21   

None

Количество дубликатов: 0


In [6]:
greeting(df_tr)

,0,1,2,3,4,5,6,7,8,9,...,63,64,65,66,67,68,69,70,71,Target
Id,,,,,,,,,,,,,,,,,,,,,
0-query,-53.882748,17.971436,-42.117104,-183.93668,187.517490,-87.144930,-347.360606,38.307602,109.085560,30.413513,...,70.107360,-155.80257,-101.965943,65.903790,34.457500,62.642094,134.763600,-415.750254,-25.958572,675816-base
1-query,-87.776370,6.806268,-32.054546,-177.26039,120.803330,-83.810590,-94.572749,-78.433090,124.915900,140.331070,...,4.669178,-151.69771,-1.638704,68.170876,25.096191,89.974976,130.589630,-1035.092211,-51.276833,366656-base
2-query,-49.979565,3.841486,-116.118590,-180.40198,190.128430,-50.837620,26.943937,-30.447489,125.771164,211.607820,...,78.039764,-169.14620,82.144186,66.008220,18.400496,212.409730,121.931470,-1074.464888,-22.547178,1447819-base
3-query,-47.810562,9.086598,-115.401695,-121.01136,94.652840,-109.255410,-775.150134,79.186520,124.003100,242.650650,...,44.515266,-145.41675,93.990981,64.131350,106.061920,83.178760,118.277725,-1074.464888,-19.902788,1472602-base
4-query,-79.632126,14.442886,-58.903397,-147.05254,57.127068,-16.239529,-321.317964,45.984676,125.941284,103.392670,...,45.028910,-196.09207,-117.626337,66.926220,42.456170,77.621765,92.479930,-1074.464888,-21.149351,717819-base


,0,1,2,3,4,5,6,7,8,9,...,62,63,64,65,66,67,68,69,70,71
count,100000.000000,100000.000000,100000.000000,100000.000000,100000.000000,100000.000000,100000.000000,100000.000000,100000.000000,100000.000000,...,100000.000000,100000.000000,100000.000000,100000.000000,100000.000000,100000.000000,100000.000000,100000.000000,100000.000000,100000.000000
mean,-85.328679,7.664345,-43.667046,-146.118630,111.770592,-73.178792,-440.615118,21.231588,123.067891,126.120201,...,-81.145553,36.898233,-152.223082,14.001695,67.799560,23.029277,73.412076,115.189717,-709.761548,-48.505704
std,25.803845,4.955651,39.111064,20.434841,47.700958,28.718853,279.017550,65.985214,6.492081,64.912752,...,30.675871,25.357686,41.020285,99.079597,1.838012,55.470761,62.203132,21.582238,405.961084,41.215124
min,-186.280270,-11.560507,-224.896060,-223.307220,-93.272020,-184.962450,-791.469482,-278.582820,94.831150,-142.156950,...,-210.059310,-60.779335,-317.831670,-157.590189,60.668890,-203.746380,-181.973820,22.598862,-1297.871984,-209.935760
25%,-103.309147,4.313784,-69.494487,-160.028520,79.474322,-92.834080,-738.536868,-22.131764,118.674642,82.545028,...,-101.504830,19.774804,-178.929470,-71.745018,66.560655,-14.144571,31.685548,100.589324,-1074.464888,-76.478591
50%,-85.273695,7.652854,-42.830246,-146.067445,112.260100,-73.364420,-511.837758,22.278989,123.087540,125.963340,...,-80.859640,36.711365,-151.576985,14.072663,67.813585,23.441363,72.880192,115.236635,-808.801696,-48.700929
75%,-67.330810,10.981191,-17.713690,-132.277745,143.760830,-53.699612,-201.116578,65.899595,127.541802,169.701005,...,-60.492451,53.919520,-124.436974,100.011024,69.058750,59.941065,114.469224,129.791085,-357.087323,-20.487869
max,14.585236,28.917845,128.108460,-60.751625,301.363600,51.849710,109.625188,288.171840,149.356200,392.827150,...,47.733450,145.667130,11.907364,185.095137,74.779884,266.493320,319.867520,201.761260,98.768233,126.191790


<class 'pandas.core.frame.DataFrame'>
Index: 100000 entries, 0-query to 99999-query
Data columns (total 73 columns):
 #   Column  Non-Null Count   Dtype  
---  ------  --------------   -----  
 0   0       100000 non-null  float64
 1   1       100000 non-null  float64
 2   2       100000 non-null  float64
 3   3       100000 non-null  float64
 4   4       100000 non-null  float64
 5   5       100000 non-null  float64
 6   6       100000 non-null  float64
 7   7       100000 non-null  float64
 8   8       100000 non-null  float64
 9   9       100000 non-null  float64
 10  10      100000 non-null  float64
 11  11      100000 non-null  float64
 12  12      100000 non-null  float64
 13  13      100000 non-null  float64
 14  14      100000 non-null  float64
 15  15      100000 non-null  float64
 16  16      100000 non-null  float64
 17  17      100000 non-null  float64
 18  18      100000 non-null  float64
 19  19      100000 non-null  float64
 20  20      100000 non-null  float64
 21  21  

None

Количество дубликатов: 0


In [7]:
greeting(df_val)

,0,1,2,3,4,5,6,7,8,9,...,62,63,64,65,66,67,68,69,70,71
Id,,,,,,,,,,,,,,,,,,,,,
100000-query,-57.372734,3.597752,-13.213642,-125.92679,110.74594,-81.279594,-461.003172,139.815720,112.880980,75.215750,...,-75.51302,52.830902,-143.43945,59.051935,69.28224,61.927513,111.59253,115.140656,-1099.130485,-117.079360
100001-query,-53.758705,12.790300,-43.268543,-134.41762,114.44991,-90.520130,-759.626065,63.995087,127.117905,53.128998,...,-79.44183,29.185436,-168.60590,-82.872443,70.76560,-65.975950,97.07716,123.391640,-744.442332,-25.009320
100002-query,-64.175095,-3.980927,-7.679249,-170.16093,96.44616,-62.377740,-759.626065,87.477554,131.270110,168.920320,...,-134.79541,37.368730,-159.66231,-119.232725,67.71044,86.002060,137.63641,141.081630,-294.052271,-70.969604
100003-query,-99.286860,16.123936,9.837166,-148.06044,83.69708,-133.729720,58.576403,-19.046660,115.042404,75.206730,...,-77.23611,44.100494,-132.53012,-106.318982,70.88396,23.577892,133.18396,143.252940,-799.363667,-89.392670
100004-query,-79.532920,-0.364173,-16.027431,-170.88495,165.45392,-28.291668,33.931936,34.411217,128.903980,102.086914,...,-123.77025,45.635944,-134.25893,13.735359,70.61763,15.332115,154.56812,101.700640,-1171.892332,-125.307890


,0,1,2,3,4,5,6,7,8,9,...,62,63,64,65,66,67,68,69,70,71
count,100000.000000,100000.000000,100000.000000,100000.000000,100000.000000,100000.000000,100000.000000,100000.000000,100000.000000,100000.000000,...,100000.000000,100000.000000,100000.000000,100000.000000,100000.000000,100000.000000,100000.000000,100000.000000,100000.000000,100000.000000
mean,-85.302233,7.669724,-43.842474,-146.119797,111.635071,-73.273042,-441.396759,21.594242,123.077725,126.001062,...,-81.129311,36.778882,-152.341249,14.397762,67.788892,23.250779,73.114446,115.196935,-709.457021,-48.416276
std,25.777321,4.956990,39.138775,20.495541,47.751576,28.515740,279.242585,66.478048,6.504931,64.981697,...,30.691590,25.427103,41.246347,98.695231,1.844522,55.403862,62.056224,21.493081,405.665764,41.292843
min,-190.353330,-11.109877,-217.538420,-220.050890,-81.198990,-176.705030,-791.460243,-265.608430,96.218350,-135.667300,...,-215.152280,-66.265970,-337.632870,-157.593541,60.377728,-210.672800,-175.921780,25.271042,-1297.923999,-209.935760
25%,-103.231790,4.292425,-69.680916,-160.210287,79.101920,-92.789803,-740.620025,-21.804730,118.657689,82.311205,...,-101.690472,19.564135,-179.115082,-70.420034,66.546130,-13.863008,31.494840,100.538862,-1074.464888,-76.359190
50%,-85.296745,7.657888,-43.230835,-146.080365,111.959330,-73.579283,-513.922978,22.138012,123.062568,126.083595,...,-81.027250,36.414150,-151.426035,14.768218,67.800445,23.508739,72.152398,115.280990,-807.029697,-48.670001
75%,-67.422104,10.993900,-17.765821,-132.119278,143.903735,-53.861139,-202.572440,66.097697,127.538967,169.680500,...,-60.274745,53.888420,-124.400438,99.803923,69.040832,60.154775,114.317053,129.770215,-358.400478,-20.283335
max,14.427986,27.409784,134.859800,-57.381890,302.065370,50.255325,109.632035,279.468900,151.822560,392.827150,...,41.390480,141.025270,14.602035,185.094333,75.400475,251.288590,305.937530,201.599980,98.737079,111.831955


<class 'pandas.core.frame.DataFrame'>
Index: 100000 entries, 100000-query to 199999-query
Data columns (total 72 columns):
 #   Column  Non-Null Count   Dtype  
---  ------  --------------   -----  
 0   0       100000 non-null  float64
 1   1       100000 non-null  float64
 2   2       100000 non-null  float64
 3   3       100000 non-null  float64
 4   4       100000 non-null  float64
 5   5       100000 non-null  float64
 6   6       100000 non-null  float64
 7   7       100000 non-null  float64
 8   8       100000 non-null  float64
 9   9       100000 non-null  float64
 10  10      100000 non-null  float64
 11  11      100000 non-null  float64
 12  12      100000 non-null  float64
 13  13      100000 non-null  float64
 14  14      100000 non-null  float64
 15  15      100000 non-null  float64
 16  16      100000 non-null  float64
 17  17      100000 non-null  float64
 18  18      100000 non-null  float64
 19  19      100000 non-null  float64
 20  20      100000 non-null  float64
 21

None

Количество дубликатов: 0


Признаки имеют разные диапазоны значений, доля пропусков крайне мала, дубликаты отсутствуют.

## Предобработка данных

Для применения FAISS требуется отсутствие пропусков. Поэтому удалим данные наблюдения, учитывая их незначительную долю.
Также масштабируем признаки - это поможет обеспечить равномерную значимость каждого признака, особенно в условиях разных диапазонов значений признаков.

In [8]:
# удаление пропусков
df_val.dropna(inplace=True)
df_tr.dropna(inplace=True)
df_base.dropna(inplace=True)

In [9]:
# Извлечение признаков
X_tr = df_tr.drop(columns=['Target']).values
Y_tr = df_tr['Target'].values
X_val = df_val.values

scaler = StandardScaler()
scaler.fit(df_base)
df_base_scaled = scaler.transform(df_base)
X_tr_scaled = scaler.transform(X_tr)
X_val_scaled = scaler.transform(X_val)

/usr/local/lib/python3.10/dist-packages/sklearn/base.py:439: UserWarning: X does not have valid feature names, but StandardScaler was fitted with feature names
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/sklearn/base.py:439: UserWarning: X does not have valid feature names, but StandardScaler was fitted with feature names
  warnings.warn(


## Приближённый поиск ближайших соседей

### FAISS

In [10]:
# создадим словарь id и номеров векторов
index_dict = {k: v for k, v in enumerate(df_base.index.to_list())}

In [11]:
# функция вычисления accuracy@5
def accuracy_n(targets, predict, index_dict=index_dict):
    accuracy = 0
    for target, similar_vectors in zip(targets.tolist(), predict.tolist()):
        accuracy += int(target in [index_dict[number] for number in similar_vectors])

    return (accuracy / len(predict) * 100)

In [12]:
# создадим индекс Faiss с использованием инвертированного индекса с IVF
dims = X_tr.shape[1]
n_cells = 500

quantizer = faiss.IndexFlatL2(dims)
idx_l2 = faiss.IndexIVFFlat(quantizer, dims, n_cells)

In [13]:
# обучение индекса и добавление векторов в индекс Faiss
idx_l2.train(np.ascontiguousarray(df_base_scaled).astype('float32'))

idx_l2.add(np.ascontiguousarray(df_base_scaled).astype('float32'))

In [14]:
# поиск ближайших соседей для векторов признаков из обучающей выборки
k = 5
distances, indices = idx_l2.search(np.ascontiguousarray(X_tr_scaled).astype('float32'), k)

In [15]:
# расчет метрики качества на тренировочном наборе данных
print('accuracy@5:', accuracy_n(Y_tr, indices))

accuracy@5: 21.029


In [16]:
# поиск ближайших соседей для векторов признаков из валидационного набора данных
k = 5
distances, indices = idx_l2.search(np.ascontiguousarray(X_val_scaled).astype('float32'), k)

In [17]:
# расчет метрики качества на валидационном наборе данных
print('accuracy@5:', accuracy_n(np.hstack(val_ans.values), indices))

accuracy@5: 20.996000000000002


## Выводы

Был использован индекс Faiss с инвертированным индексом и векторными фильтрами (IVF).

Предварительно признаки были масштабированы, удалены пропуски.

Accuracy@5 на обучающей и валидационной выборках - 21%. Полученные результаты показывают, что модель предсказывает наиболее похожих соседей для примерно каждого пятого товара из обучающей и валидационной выборок.
